transformers: 사전학습된 NLP 모델과 토크나이저 제공 라이브러리

  - AutoTokenizer: 모델 적합 토크나이저(text->token ID) 자동 로딩

  - AutoModelForCausalLM: CausalLanguageModeling(자연어 생성, **왼->오 순차적 단어 예측 및 생성**) 적합 모델 자동 로딩

peft: 유효한 파라미터 파인 튜닝을 돕는 모듈 라이브러리. Prefix 튜닝도 PEFT 기법 中 하나

  - get_peft_model: PEFT 설정을 통해 모델에 적용되는(감싸는) 함수

  - PrefixTuningConfig: Prefix Tuning 설정을 위한 구성 클래스

  - TaskType: 작업 유형(ex.Causal_LM, Seq_CLS)에 따라 적용 PEFT 결정

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, PrefixTuningConfig, TaskType

# KoGPT-2 모델 및 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name) # 모델 적합 토크나이저 자동 로드
model = AutoModelForCausalLM.from_pretrained(model_name) # causalLM 작업에 적절한 모델(KoGPT2) 로드

# pad_token이 없으면 추가
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # 새롭게 PAD 토큰 추가
    model.resize_token_embeddings(len(tokenizer))         # 모델 vocab 크기도 늘려줌


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


**Pad Token**

=> Prefix 튜닝에서 padding(여러 문장 입력 시, **가장 긴 문장을 기준으로 입력 길이 통일**) 요구됨.


---


ex. 입력 문장 토큰 길이=5 지정 시,

["나는", "간다", "[PAD]", "[PAD]", "[PAD]"]

attention_mask = [1, 1, 0, 0, 0]

=> [PAD] 는 무시하고 유효한 실제 토큰만 계산

---



=> GPT 계열 모델에서 PAD 토큰이 없는 경우 多

=> Pad 토큰 직접 추가 -> 모델 vocab 크기 늘어남 -> 모델 embedding 테이블 크기(embedding layer)도 늘려줘야 함.

In [ ]:
# Prefix 튜닝 세팅
prefix_config = PrefixTuningConfig(
    task_type=TaskType.CAUSAL_LM, # Task 종류(Causal_LM) 지정
    num_virtual_tokens=25, # Prefix 토큰 수 지정
    prefix_projection=True, # Prefix 토큰 선형(Projection layer) 변환 여부
    inference_mode=False # True: 학습없이 생성만, False: 학습모드
)
model = get_peft_model(model, prefix_config) # Prefix 튜닝 구조로 모델 적용(감싸기)

내부구조

=> [Input Text] → [Prefix Embeddings (10개)] → [KoGPT2 (고정)] → [Output]

**List comperhension** (반복되는 객체를 새로운 리스트로 생성)

  - [표현식 for 변수 in 반복 가능한 객체]


  - ex. numbers = [x * 2 for x in range(5)] → [0, 2, 4, 6, 8]


In [ ]:
import pandas as pd

# 학습용 정제된 시, 발라드 데이터 로드(200, 100개씩 제한)
poem_df = pd.read_csv("poems_final_pp_4040.csv",encoding="utf-8-sig")
# ballard_df = pd.read_csv("Ballard_Lyrics_Cleaned_517.csv", encoding="utf-8-sig")
indie_df = pd.read_csv("Indie_Lyrics_Cleaned_567.csv", encoding="utf-8-sig")
# hiphop_df = pd.read_csv("HipHop_Lyrics_Cleaned_600.csv", encoding="utf-8-sig")

# list comperhension(반복되는 객체를 새로운 리스트로 생성)
# 시 데이터셋을 poems_texts로 구성
poem_texts = [f"제목: {row['제목']}\n내용: {row['내용']}" for _, row in poem_df.iterrows()]
# _ : 사용하지 않는 변수에 대해 생략의 의미
# df.iterrows(): df의 한 행씩 반복 => (index, row Series data)
# 발라드/인디/힙합 장르별 가사 데이터셋을 ()_texts로 구성
# ballard_texts = [f"제목: {row['제목']}\n가사: {row['가사']}" for _, row in ballard_df.iterrows()]
indie_texts = [f"제목: {row['제목']}\n가사: {row['가사']}" for _, row in indie_df.iterrows()]
# hiphop_texts = [f"제목: {row['제목']}\n가사: {row['가사']}" for _, row in hiphop_df.iterrows()]

In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split

poem_train, poem_test = train_test_split(poem_texts, test_size=0.2, random_state=42)
# ballard_train, ballard_test = train_test_split(ballard_texts, test_size=0.2, random_state=42)
indie_train, indie_test = train_test_split(indie_texts, test_size=0.2, random_state=42)
# hiphop_train, hiphop_test = train_test_split(hiphop_texts, test_size=0.2, random_state=42)

# 시 데이터 리스트와 발라드 가사 데이터 리스트 합치기
# combined_train_texts = poem_train + ballard_train
# combined_test_texts = poem_test + ballard_test
# 시 데이터 리스트와 인디 가사 데이터 리스트 합치기
combined_train_texts = poem_train + indie_train
combined_test_texts = poem_test + indie_test
# 시 데이터 리스트와 힙합 가사 데이터 리스트 합치기
# combined_train_texts = poem_train + hiphop_train
# combined_test_texts = poem_test + hiphop_test

# Train / Test Dataset 객체 생성
train_dataset = Dataset.from_dict({"text": combined_train_texts})
test_dataset = Dataset.from_dict({"text": combined_test_texts})

토크나이저 사용

-> "key:text" 일 때 토크나이저가 데이터로 인식함.

-> input_ids(입력 데이터의 벡터 인덱스) 자동 생성

-> input_ids를 기반으로 labels(예측해야하는 데이터의 벡터 인덱스) 세팅이 가능함.

In [ ]:
# 토크나이징
def tokenize(example):
    enc = tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)
    # truncation=True : 최대 토큰 길이보다 길면 자름
    # 문장 최대 토큰 길이 = max_length(256) -> PAD 토큰 사용의 기준
    enc["labels"] = enc["input_ids"].copy()
    return enc

# 전체 Dataset(train/test 모두) 토크나이징 적용
tokenized_train = train_dataset.map(tokenize)
tokenized_test = test_dataset.map(tokenize)
# batched=True(text 데이터를 리스트로 입력받아 빠르게 학습, 보편적으로 성능 높아짐) => 추후 시도해볼것

Map:   0%|          | 0/3685 [00:00<?, ? examples/s]

Map:   0%|          | 0/922 [00:00<?, ? examples/s]

***dataset 데이터셋 구조***

DatasetDict(300)({

  train: Dataset({

    features: ['text'],
    
    num_rows: 240
  
  }),
  
  test: Dataset({
    
    features: ['text'],
    
    num_rows: 60
  
  })

})

***tokenized 데이터셋 구조***

DatasetDict(300)({

  train: Dataset({

    features: ['input_ids', 'attention_mark', 'labels'],
    
    num_rows: 240
  
  }),
  
  test: Dataset({
    
    features: ['input_ids', 'attention_mask', 'labels'],
    
    num_rows: 60
  
  })

})

dataset["train"][0]["text"]  => 실제 데이터 확인

tokenized["train"][0]["input_ids"]  => 토큰 ID(입력 데이터의 벡터 변환값) 확인

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./kogpt_skt_results", # 학습 결과물 저장할 디렉토리 지정
    num_train_epochs=5, # 학습 에폭 수(= 전체 데이터 반복학습 횟수), 일반적 3~5
    per_device_train_batch_size=8, # 한번에 학습할 데이터 개수
    per_device_eval_batch_size=8,
    learning_rate=5e-6, # 안정적 학습률 추가
    logging_strategy="steps", # Train loss log 남기기 위함
    logging_steps=10, # 기록할 스텝 단위
    logging_dir="./logs",
    eval_strategy = "epoch", # logging_steps 마다 평가 진행
    save_total_limit=1, # 체크포인트 저장 개수
    save_strategy="epoch", # epoch 마다 모델 저장, 최종적으로 가장 성능 좋은 모델 1개 유지
    load_best_model_at_end=False,
    # Trainer: 전체 모델 weight 포함 저장, peft: Prefix adapter만 저장 -> mismatch -> 모델 수동 로드
    report_to="none"
)

# Trainer(AdamW 옵티마이저 사용) 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipython-input-3446922738.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,7.285600,7.268509
2,7.563400,7.186357
3,7.690700,7.154195
4,7.307400,7.139791
5,7.469100,7.135609


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

TrainOutput(global_step=2305, training_loss=7.362219168772667, metrics={'train_runtime': 1116.481, 'train_samples_per_second': 16.503, 'train_steps_per_second': 2.065, 'total_flos': 2407152844800000.0, 'train_loss': 7.362219168772667, 'epoch': 5.0})

In [ ]:
# evaluation loss 기반 최적의 checkpoint 모델 찾기
# log_history 중 eval_loss 있는 것만 추출
eval_logs = [log for log in trainer.state.log_history if 'eval_loss' in log]

# eval_loss 기준 가장 낮은 로그 탐색
best_model_log = min(eval_logs, key=lambda x: x['eval_loss'])

# 해당 스텝
best_model_step = best_model_log['step']

# 해당 checkpoint 경로
best_ckpt_path = f"./kogpt_skt_results/checkpoint-{best_model_step}"
print(f"최적 모델 경로(checkpoint): {best_ckpt_path}")

최적 모델 경로(checkpoint): ./kogpt_skt_results/checkpoint-2305


모델 평가 지표

- Training Loss: Train data set에 대한 모델 학습 정도(낮을수록 Train data set에 대해 잘 학습된 모델)

- Evaluation Loss: Validation data set에 대한 모델 예측 정도(낮을수록 일반화 성능 굿)

- Perplexity: 생성 결과의 혼란도 측정(낮을수록 문맥 예측 성능 굿)

In [ ]:
# 가장 낮은 eval_loss 모델과 토크나이저 로드
from peft import PeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(best_ckpt_path)

base_model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")
base_model.resize_token_embeddings(len(tokenizer))
model = PeftModelForCausalLM.from_pretrained(base_model, best_ckpt_path)

In [ ]:
import torch
import math
from torch.utils.data import DataLoader

# best checkpoint 모델 기반 trainer 재정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer
)

trainer.train(resume_from_checkpoint=None)

# evaluation 수행 및 evaluation loss, Perplexity 계산
eval_result = trainer.evaluate()
eval_loss = eval_result["eval_loss"]
perplexity = math.exp(eval_loss)

# trainer의 log_history에서 최종 training loss 추출
train_loss_log = [log["loss"] for log in trainer.state.log_history if "loss" in log and "eval_loss" not in log]
final_train_loss = train_loss_log[-1] if train_loss_log else None

print(f"[최종 모델 성능]")
print(f"Training Loss:  {final_train_loss:.4f}")
print(f"Evaluation Loss:  {eval_loss:.4f}")
print(f"Perplexity(PPL):  {perplexity:.4f}")

/tmp/ipython-input-2366283674.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,7.106100,7.135407
2,7.494300,7.135236
3,7.668700,7.135110
4,7.300200,7.135033
5,7.468300,7.135008


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

[최종 모델 성능]
Training Loss:  7.4683
Evaluation Loss:  7.1350
Perplexity(PPL):  1255.1475


Trainer는 내부적으로

- DataLoader 구성
- Optimizer 생성
- input_ids / attention_mask / labels 세 가지 키 자동 확인 후 loss 계산
- eval_dataset 존재 시, 평가 수행


---

*About Hugging Face, Trainer, TrainingArguments*

: https://sangwonyoon.tistory.com/entry/HuggingFace-Trainer%EB%A1%9C-%EB%AA%A8%EB%8D%B8-%ED%95%99%EC%8A%B5-%EB%B0%8F-%ED%8F%89%EA%B0%80%ED%95%98%EA%B8%B0

In [ ]:
import torch

input_title = "제목: 작은 연못\n내용:" # 명시적 프롬프트 제시
inputs = tokenizer(input_title, return_tensors="pt", padding=True) # "pt" : PyTorch 텐서 형식으로 변환
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 입력 텐서를 모두 동일한 디바이스(GPU)로 이동
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
model = model.to(device)

output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=100,
    do_sample=True, # 샘플링 기반 랜덤 생성
    temperature=0.8, # 확률 분포 정도 조절(높을수록 창의적)
    top_p=0.9, # 누적 확률 p(여기서는 0.9) 이하까지만 후보로 고려
    top_k=50, # 상위 k개 토큰만 고려
    pad_token_id=tokenizer.pad_token_id, # PAD 토큰 없는 경우 강제 추가
    repetition_penalty=1.2 # 반복 방지 추가
) # 모델이 생성한 토큰 ID(백테 변환값)

print(tokenizer.decode(output[0], skip_special_tokens=True))
# skip_special_tokens=True : PAD, EOS 토큰 생략


제목: 작은 연못
내용: 작은 연못에서 만난 연인들
연인과의 추억이 담긴 이 아름다운 연꽃의 향기
그들의 아름다움으로 가득한 물소리 속에
순백의 꽃향기에 반한 채
더러운 꿈속에서 맴돌던 사랑의 불빛이
정말 행복했던 것 같다
지난 여름의 한 달보다 더 행복한 날들이 많았다
그리고 그렇게 정말로 소중한 기억들은 오래도록 남아 있다
오늘도 그대의 사랑스런 모습을 그대에게
잊을 수 없을 것만 같아요
사랑스럽고, 아름답게 빛나는


# 생성 결과 비교 및 평가를 위한 Embedding

In [ ]:
# 한국어 문장 임베딩 모델 설치 및 로드
!pip -q install sentence-transformers scikit-learn # 한번만 실행

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.1 MB/s eta 0:00:00


In [ ]:
# 임베딩 모델 로드
from sentence_transformers import SentenceTransformer
# 한국어에 강한 대표 모델
# m_name = "BM-K/KoSimCSE-roberta-multitask"
m_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
emb_model = SentenceTransformer(m_name)

"""
왜 Sentence-BERT류인가?
KoGPT-2의 은닉상태 평균풀링도 가능하지만,
문장 의미 유사도 평가는 보통 SBERT/SimCSE 계열 훨씬 안정적
(뒷받침 근거 논문 찾기 필요)
"""

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

'\n왜 Sentence-BERT류인가?\nKoGPT-2의 은닉상태 평균풀링도 가능하지만,\n문장 의미 유사도 평가는 보통 SBERT/SimCSE 계열 훨씬 안정적\n(뒷받침 근거 논문 찾기 필요)\n'

In [ ]:
# 생성 결과 유사도 평가 대상(코퍼스) 선택 및 포맷 정리
import re

"""
평가 대상 선택
훈련 시 데이터만 사용.
가사 데이터 함께 적용 시(combined_train_texts),
유사도 기준에 장르적 혼돈 영향을 끼칠 수 있음.
"""
ref_corpus = poem_train

# 데이터 포맷 정규화(공백, 제어문자 정리)
def normalize_text(s: str) -> str:
    s = s.strip()
    s = re.sub(r'\s+', ' ', s)
    return s

ref_corpus_norm = [normalize_text(t) for t in ref_corpus]

In [ ]:
# 코퍼스(시 데이터 4040개) 한 번 임베딩 후 .npy 저장 및 재사용
import numpy as np
from pathlib import Path

cache_dir = Path("./emb_cache")
cache_dir.mkdir(exist_ok=True)
emb_path = cache_dir / f"{m_name.replace('/', '_')}_poem_train_emb_blank.npy" # 임베딩 결과 저장
txt_path = cache_dir / f"{m_name.replace('/', '_')}_poem_train_texts_blank.txt"

def encode_texts(texts):
    return emb_model.encode(
        texts,
        batch_size=64,
        convert_to_numpy=True,
        normalize_embeddings=True, # L2 정규화 => 코사인 = 내적
        show_progress_bar=True
    )

if emb_path.exists() and txt_path.exists():
    ref_embs = np.load(emb_path)
    with open(txt_path, "r", encoding="utf-8") as f:
        ref_corpus_norm = [line.rstrip("\n") for line in f]
else:
    ref_embs = encode_texts(ref_corpus_norm)
    np.save(emb_path, ref_embs)
    with open(txt_path, "w", encoding="utf-8") as f:
        for t in ref_corpus_norm:
            f.write(t + "\n")

ref_embs.shape  # ex. (N, 768)

Batches:   0%|          | 0/51 [00:00<?, ?it/s]

(3232, 768)

In [ ]:
# 기존 단일 프롬프트 생성 -> 함수(wrapper) 적용하여 여러 프롬프트 배치 생성
import torch

def generate_poem(prompt_title: str,
                  max_new_tokens=120,
                  temperature=0.8,
                  top_p=0.9,
                  top_k=50,
                  repetition_penalty=1.2):

    if not prompt_title.startswith("제목:"):
        prompt = f"제목: {prompt_title}\n내용:" # 생성 프롬프트 포맷(명시적 프롬프트)과 통일하기 위함
    else:
        prompt = prompt_title

    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            pad_token_id=tokenizer.pad_token_id,
            repetition_penalty=repetition_penalty
        )

    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return text

def split_title_body(generated_text: str):
    title = ""
    body = generated_text
    if "내용:" in generated_text:
        parts = generated_text.split("내용:", 1)
        title = parts[0].replace("제목:", "").strip()
        body = parts[1].strip()
    return title, body # 기존 "제목: \n내용:" 포맷에서 title, body 각각 분리

평가 지표

- sim_top1: 생성 시와 가장 가까운 훈련 시 1편과의 유사도. 높을수록(ex.0.9 이상) 훈련 시 데이터에 대한 오버피팅 확률 큼.

- sim_top5_mean: 상위 5편 평균 유사도

- novelty = 1 - sim_top1: 신생성(참신함) 대략치

- 가장 가까운 훈련 시 Top-K(제목/내용 일부)로 질적 점검

In [ ]:
# 임베딩 유사도 평가(코사인 유사도)
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def embed_text(text: str) -> np.ndarray:
    vec = emb_model.encode([normalize_text(text)],
                           convert_to_numpy=True,
                           normalize_embeddings=True)
    return vec[0]

def evaluate_generated_text(gen_text: str, ref_texts: list, ref_embs: np.ndarray, k: int = 5):
    """
    gen_text: "제목:\n내용:" 프롬프트 포함된 시 생성 결과
    ref_texts: ref_corpus_norm와 동일한 길이/순서의 훈련 시 데이터
    ref_embs: 훈련 시 데이터 임베딩 벡터 변환 값
    """
    g = embed_text(gen_text)
    # ref_embs가 L2 normalized라면 내적 == 코사인
    sims = ref_embs @ g

    # 코사인 유사도 기준 Top-K 인덱스 추출
    top_idx = np.argsort(-sims)[:k]
    top_sims = sims[top_idx]

    result = {
        "sim_top1": float(top_sims[0]),
        "sim_top5_mean": float(top_sims.mean()),
        "novelty": float(1.0 - top_sims[0]),
        "top_indices": top_idx.tolist(),
        "top_sims": top_sims.tolist(),
        "top_texts": [ref_texts[i] for i in top_idx]
    }
    return result

# 여러 개 프롬프트 동시 평가
prompts = ["초록빛 향연", "작은 연못", "하얀", "고운", "하얀 마음", "당신의 고운 노래"]
records = []

for p in prompts:
    gen = generate_poem(p) # 명시적 프롬프트 포맷 자동 적용
    title, body = split_title_body(gen)

    # 평가 단위: 전체 포맷로 비교
    # gen_for_eval = normalize_text(gen)
    # 평가 단위: 내용만 비교
    # 본 연구의 경우 생성 결과가 내용이기 때문에, 내용만 유사도 비교하는게 어떨지
    gen_for_eval = normalize_text(body)

    eval_res = evaluate_generated_text(gen_for_eval, ref_corpus_norm, ref_embs, k=5)

    # 가장 유사한 훈련 시 데이터 1개와 질적 비교
    nn_text = eval_res["top_texts"][0]
    nn_title, nn_body = split_title_body(nn_text)
    nn_excerpt = (nn_body[:120] + "…") if len(nn_body) > 120 else nn_body

    records.append({
        "prompt": p,
        "gen_title": title,
        "gen_body": body,
        "sim_top1": eval_res["sim_top1"],
        "sim_top5_mean": eval_res["sim_top5_mean"],
        "novelty": eval_res["novelty"],
        "nn_title": nn_title,
        "nn_excerpt": nn_excerpt
    })

df_eval = pd.DataFrame(records)
df_eval.sort_values("sim_top1", ascending=False, inplace=True)
df_eval.reset_index(drop=True, inplace=True)
df_eval.to_csv("compare_generation_by_embedding.csv", index=False, encoding="utf-8-sig")